In [ ]:
!pip install selenium

     |████████████████████████████████| 954 kB 12.1 MB/s 
     |████████████████████████████████| 138 kB 87.1 MB/s 
     |████████████████████████████████| 356 kB 87.7 MB/s 
     |████████████████████████████████| 3.5 MB 80.5 MB/s 
     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
# coding: utf-8
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from time import sleep
from PIL import Image

import datetime
import requests
import random
import json
import os


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
logins = [{"email":[email-1], "senha":[senha-1]},
          {"email":[email-2], "senha":[senha-2]}]


In [ ]:
def get_credentials():
  return logins[random.randint(0,len(logins)-1)]


In [ ]:
def make_login():
    driver = webdriver.Chrome(executable_path=os.path.abspath("chromedriver.exe"))
    driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

    current_login = get_credentials()
    username = driver.find_element_by_id('username')
    username.send_keys(current_login["email"])

    password = driver.find_element_by_id('password')
    password.send_keys(current_login["senha"])

    log_in_button = driver.find_element_by_xpath('//*[@type="submit"]')
    log_in_button.click()

In [ ]:
def parse_html(to_parse):
    name = to_parse.get_attribute('outerHTML')
    return BeautifulSoup(name, "html.parser")

In [ ]:
def save_image(image_url, name):
    name = name.replace(' ', '_')
    file_name = 'profiles/{0}.jpg'.format(name)
    img = Image.open(requests.get(image_url, stream = True).raw)
    img.save(file_name)

    return file_name

In [ ]:
def get_profile_json(profile):
    driver.get(profile)

    try:
        see_more_skills_button = driver.find_element_by_class_name('artdeco-empty-state')
        current_login = current_login + 1
    except:
        pass

    height_initial = (driver.find_element_by_id("profile-content")).size["height"]
    height_current = 0


    more_about = False
    more_experience = False
    more_skills = False

    while height_current < height_initial:
        driver.execute_script("window.scrollTo(0, {0})".format(height_current))
        height_current = height_current + 100
        sleep(0.5)

        if not more_about:
            try:
                see_more_about_button = driver.find_element_by_class_name('inline-show-more-text__link-container-collapsed')
                see_more_about_button.click()

                more_about = True
            except:
                pass

        while not more_experience:
            try:
                see_more_experience_button = driver.find_element_by_class_name('pv-profile-section__see-more-inline')
                btn_text = parse_html(see_more_experience_button).get_text()

                if not 'fewer' in btn_text and not 'less' in btn_text:
                    see_more_experience_button.click()
                else:
                    more_experience = True
            except:
                more_experience = True
                pass

        if not more_skills:
            try:
                see_more_skills_button = driver.find_element_by_class_name('pv-skills-section__additional-skills')
                see_more_skills_button.click()
                more_skills = True
            except:
                continue

    driver.execute_script("window.scrollTo(0, 0)")

    #name
    soup = parse_html(driver.find_element_by_class_name("pv-top-card"))
    try:
        name = soup.find('h1').get_text()
    except:
        name = ''

    #picture
    try:
        picture = soup.find('img', class_='presence-entity__image')
        picture = save_image(picture["src"], name.lower())
    except:
        picture = ''

    #locate
    try:
        locate = (soup.find('span', class_='text-body-small inline t-black--light break-words').get_text()).strip()
    except:
        locate = ''

    #work
    try:
        work = (soup.find('div', class_='text-body-medium break-words').get_text()).strip()
    except:
        work = ''

    #Open
    try:
        open_to_work = soup.find('div', class_='pv-open-to-carousel-card__content').h3.get_text()
        open_to_work = True
    except:
        open_to_work = False

    #info
    driver.execute_script("window.scrollTo(0, 0)")
    try:
        info = parse_html(driver.find_element_by_class_name("profile-detail"))
        info = (info.find('div', class_='inline-show-more-text').get_text()).strip()
    except:
        info = ''

    experiences = driver.find_elements_by_class_name('pv-entity__position-group-pager')
    educations = driver.find_elements_by_class_name('pv-education-entity')
    certifications = driver.find_elements_by_class_name('pv-certification-entity')
    skills = driver.find_elements_by_class_name('pv-skill-category-entity')
    depoiments = driver.find_elements_by_class_name('pv-recommendation-entity')
    languages = driver.find_elements_by_class_name('languages li')

    #experience
    list_experiences = []
    for exp in experiences:
        _experience = {"title":"", "company":"", "duration":""}
        content = exp.get_attribute('outerHTML')
        exp = BeautifulSoup(content, "html.parser")

        exp_title = exp.find('h3', class_='t-14')
        if not exp_title:
            exp_title = exp.find('h3', class_='t-16')

        exp_company = exp.find('p', class_='pv-entity__secondary-title')
        if not exp_company:
            exp_company = exp.find('div', class_='pv-entity__company-summary-info').h3

        exp_duration = exp.find('span', class_='pv-entity__bullet-item-v2')
        try:
            if not exp_duration:
                exp_duration = exp.find('div', class_='pv-entity__company-summary-info').h4
        except:
            pass

        description = ''
        try:
            exp_description = exp.find('div', class_='pv-entity__description')
            description = exp_description.get_text().replace('see more', '')
            description = description.replace('see less', '')
            description = description.replace('\n', '')
            description = description.replace('  ', '')
            description = description.replace('\t', '')
            description = description.replace('See more','')
            description = description.replace('     ','')
            description = description.replace('see less\n        \n\n','')
            description = description.replace('see more\n        \n\n','')

        except:
            pass
        _experience["description"] = description

        title = exp_title.get_text()
        title = title.replace('Title','').strip()
        title = title.replace('\n','').strip()
        _experience["title"] = title

        company = exp_company.get_text().strip()
        company = company.replace('Company Name','').strip()
        company = company.replace('\n','').strip()
        company = company.replace('        ',' ').strip()

        _experience["company"] = company

        try:
            duration = exp_duration.get_text().strip()
            duration = duration.replace('Total Duration','').strip()
            duration = duration.replace('\n','').strip()
            _experience["duration"] = duration
        except:
            pass


        list_experiences.append(_experience)


    #Education
    list_educations = []
    for edc in educations:
        _education = {"school":"", "course":"", "duration":""}
        content = edc.get_attribute('outerHTML')
        edc = BeautifulSoup(content, "html.parser")

        edc_title = edc.find('h3', class_='pv-entity__school-name t-16 t-black t-bold')
        if edc_title:
            _education["school"] = edc_title.get_text()

        edc_field = edc.find('p', class_='pv-entity__secondary-title pv-entity__fos t-14 t-black t-normal')
        if edc_field:
            edc_field = edc_field.get_text()
            edc_field = edc_field.replace('Field Of Study', '')
            edc_field = edc_field.replace('\n', '')
            _education["course"] = edc_field

        edc_course = edc.find('p', class_='pv-entity__secondary-title pv-entity__degree-name t-14 t-black t-normal')
        if edc_course:
            edc_course = edc_course.get_text()
            edc_course = edc_course.replace('Degree Name', '')
            edc_course = edc_course.replace('\n', '')
            _education["course"] = "{0} - {1}".format(edc_course, _education["course"])

        edc_duration = edc.find('p', class_='pv-entity__dates t-14 t-black--light t-normal')
        if edc_duration:
            edc_duration = edc_duration.get_text()
            edc_duration = edc_duration.replace('Dates attended or expected graduation', '')
            edc_duration = edc_duration.replace('\n', '')
            _education["duration"] = edc_duration

        list_educations.append(_education)

    #Course
    list_certifications = []
    for cert in certifications:
        _certification = {"school":"", "course":"", "duration":"", "credential":""}
        content = cert.get_attribute('outerHTML')
        cou = BeautifulSoup(content, "html.parser")

        cou_title = cou.find('div', class_='pv-certifications__summary-info pv-entity__summary-info pv-entity__summary-info--background-section pv-certifications__summary-info--has-extra-details')
        if cou_title:
            cou_title = cou_title.get_text()
            cou_title = cou_title.replace('\n', '').replace('    ', '').replace('  ', ' - ')
            cou_title = cou_title.replace('No Expiration DateCredential Identifier', '')
            cou_title = cou_title.replace('Issued date and, if applicable, expiration date of the certification or license', '')
            cou_title = cou_title.replace('Issuing authority', '')
            cou_title = cou_title.replace('-   -', ' - ').replace('-  -', ' - ')

            cou_titles = cou_title.split(' - ')

            _certification["course"] = cou_titles[0]
            try:
                _certification["school"] = cou_titles[1]
                _certification["duration"] = cou_titles[2]
                _certification["credential"] = cou_titles[3]
            except:
                pass

            list_certifications.append(_certification)
    #skill
    list_skills = []
    for skill in skills:
        _skill = {"skill":"", "votes":""}
        content = skill.get_attribute('outerHTML')
        skill = BeautifulSoup(content, "html.parser")

        skill_title = skill.find('span', class_='pv-skill-category-entity__name-text t-16 t-black t-bold')
        if skill_title:
            skill_title = skill_title.get_text().strip()
            skill_title = skill_title.replace('\n','')
            _skill["skill"] = skill_title

        skill_votes = skill.find('span', class_='pv-skill-category-entity__endorsement-count t-14 t-black--light t-normal')
        if skill_votes:
            skill_votes = skill_votes.get_text()
            _skill["votes"] = skill_votes

        list_skills.append(_skill)

    #depoiment
    list_depoiments = []
    for depoiment in depoiments:
        _depoiment = {"author":"", "depoiment":""}
        content = depoiment.get_attribute('outerHTML')
        depoiment = BeautifulSoup(content, "html.parser")



        depoiment_author = depoiment.find('div', class_='pv-recommendation-entity__detail')
        if depoiment_author:
            depoiment_author = depoiment_author.h3.get_text().strip()
            depoiment_author = depoiment_author.replace('\n','')
            _depoiment["author"] = depoiment_author

        depoiment_depoiment = depoiment.find('div', class_='pv-recommendation-entity__highlights')
        if depoiment_depoiment:
            depoiment_depoiment = depoiment_depoiment.get_text().strip()
            depoiment_depoiment = depoiment_depoiment.replace('\n','')
            depoiment_depoiment = depoiment_depoiment.replace('...','')
            depoiment_depoiment = depoiment_depoiment.replace('See more','')
            depoiment_depoiment = depoiment_depoiment.replace('     ','')
            depoiment_depoiment = depoiment_depoiment.replace('see less\n        \n\n','')
            depoiment_depoiment = depoiment_depoiment.replace('see more\n        \n\n','')


            _depoiment["depoiment"] = depoiment_depoiment

        list_depoiments.append(_depoiment)

    #languages
    list_languages = []
    for language in languages:
        _language = {"language":""}
        content = language.get_attribute('outerHTML')
        language = BeautifulSoup(content, "html.parser")

        if language:
            language_item = language.get_text().strip()

            _language["language"] = language_item

            list_languages.append(_language)

    data = {"name":name, "picture":picture, "created_at":datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "profile":profile, "open_to_work":open_to_work, "locate":locate, "work":work, "info":info, "experiences":list_experiences, "educations":list_educations, "courses":list_certifications, "skills":list_skills, "depoiments":list_depoiments, "languages":list_languages}

    file_name = 'profiles/{0}.json'.format(name.lower().replace(" ", "_").replace(".", ""))
    with open(file_name, 'w') as outfile:
        json.dump(data, outfile)

In [ ]:
make_login()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


WebDriverException: ignored

In [ ]:
with open("lista_profiles.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

keep_searching = True
profiles = soup.find_all('a', {'class' : 'ember-view ui-conditional-link-wrapper ui-entity-action-row__link'})
for profile_link in profiles:
    if profile_link["href"] == [profile-link]
        keep_searching = False

    try:
        if not keep_searching:
            print(profile_link["href"])
            get_profile_json(profile_link["href"])
    except:
        continue

In [ ]:
get_profile_json([profile-link])

In [ ]:
profs = open("profiles.json", encoding='utf-8')
profs = json.load(profs)

count_requests = 0

for prof in profs:
    get_profile_json(prof)
    count_requests += 1

    if count_requests == 100:
        count_requests = 0
        make_login()
